In [4]:
import lmdb
import numpy as np
import os
import pickle
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd

In [12]:
def read_lmdb(lmdb_path):
    env = lmdb.open(
        lmdb_path,
        subdir=False,
        readonly=True,
        lock=False,
        readahead=False,
        meminit=False,
        max_readers=256,
    )
    list_data = []
    txn = env.begin()
    keys = list(txn.cursor().iternext(values=False))
    for idx in keys:
        datapoint_pickled = txn.get(idx)
        data = pickle.loads(datapoint_pickled)
        list_data.append(data)
    return list_data

In [13]:
lmdb_path = "data_split/split_1.lmbd"
data = read_lmdb(lmdb_path)

print(data[0])
results = {
    "smi": [],
    "num_atoms": [],
    "num_conf": [],
    "can_RD": [],
}

bad_smi = {
     "smi": [],
    "num_atoms": [],
    "num_conf": [],
}
for idx, dp in enumerate(data):
    can_RD = True if Chem.MolFromSmiles(dp["smi"]) is not None else False

    results["smi"].append(dp["smi"])
    results["num_atoms"].append(len(dp["atoms"]))
    results["num_conf"].append(len(dp["coordinates"]))
    results["can_RD"].append(can_RD)

    bad_smi["smi"].append(dp["smi"])
    bad_smi["num_atoms"].append(len(dp["atoms"]))
    bad_smi["num_conf"].append(len(dp["coordinates"]))

# temp = pd.DataFrame(results)
# temp.to_csv("test.csv")
# temp = pd.DataFrame(bad_smi)
# temp.to_csv("test2.csv")


{'atoms': ['C', 'O', 'C', 'C', 'Cl', 'C', 'C', 'N', 'C', 'O', 'N', 'C', 'C', 'C', 'S', 'C', 'C', 'O', 'C', 'C', 'Cl', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'], 'coordinates': [array([[ 5.7474146 , -0.42737713, -0.671256  ],
       [ 5.0595884 , -0.873312  ,  0.49939108],
       [ 3.7335892 , -0.5272272 ,  0.44309688],
       [ 3.2894847 ,  0.6755131 ,  0.9917155 ],
       [ 4.3869643 ,  1.7868254 ,  1.733296  ],
       [ 1.9271951 ,  1.0098677 ,  0.9667565 ],
       [ 0.96455485,  0.14713344,  0.41510382],
       [-0.42406547,  0.49487233,  0.39038765],
       [-0.94065493,  1.7353208 ,  0.68675125],
       [-0.37511313,  2.7829528 ,  0.9546822 ],
       [-2.3129926 ,  1.6798112 ,  0.61305684],
       [-2.7886627 ,  0.43308115,  0.0777128 ],
       [-3.8673182 , -0.19075151,  0.95417225],
       [-4.5347543 , -1.2939454 ,  0.15051603],
       [-4.67886   , -0.6208725 , -1.5233577 ],
       [-3.331029  ,  0.5865149 , -1.3439028 ],
       [-1.4680768 , -0.3581401 ,  0.

/tmp/ipykernel_28440/67900868.py:16: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.loads(datapoint_pickled)


In [31]:
env = lmdb.open("example_data/molecule/SmiGone/train.lmdb", subdir=False, lock=False)
with env.begin(write=True) as txn:
    for idx, data in enumerate(data):
        del data["smi"] 
        key = f"{idx}".encode("ascii")
        txn.put(key, pickle.dumps(data))


In [22]:
# env = lmdb.open(
#         lmdb_path,
#         subdir=False,
#         readonly=True,
#         lock=False,
#         readahead=False,
#         meminit=False,
#         max_readers=256,
#     )
idx = 505
datapoint_pickled = env.begin().get(f"{idx}".encode("ascii"))
print(datapoint_pickled)

b'\x80\x04\x95\xfc \x00\x00\x00\x00\x00\x00}\x94(\x8c\x05atoms\x94]\x94(\x8c\x01C\x94h\x03h\x03h\x03h\x03\x8c\x01N\x94h\x03h\x03\x8c\x01O\x94h\x04h\x03h\x03h\x03h\x03h\x03\x8c\x01F\x94h\x03h\x03h\x03h\x03h\x03h\x05h\x04h\x03h\x03h\x03h\x03\x8c\x01S\x94h\x03h\x05h\x05h\x03h\x03\x8c\x02Cl\x94\x8c\x01H\x94h\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\th\te\x8c\x0bcoordinates\x94]\x94(\x8c\x16numpy._core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01KAK\x03\x86\x94h\x0f\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x0c\x03\x00\x00\xf1\x13R@\xd6\xf1\x84@\xdav2\xc0tEA@\'\xc0D@#r\xe3\xbf\xf5\x89\x0c@? V@\x12l-\xbf\x82\xdb\xf5?w\xb5\x16@\xc8b\x89>\xa7\x9e\x1d@\xf8\xb3\x89?\xda?\n>Z\\\x0c@\xeeD\xc5=\xcd\x06\x94?->\xfc?\x9c"\xa9\xbf\xce\x98E?\xb8\xcen?\xc7\x8d\xbf\xbf\xa6\xfc\xa7\xbe\x14

In [6]:
import torch

# Load checkpoint
checkpoint = torch.load("mol_pre_no_h_220816.pt", map_location='cpu')

# If it's a state_dict (common case)
if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
elif isinstance(checkpoint, dict):
    state_dict = checkpoint
else:
    raise ValueError("Unexpected checkpoint format")

print(state_dict["model"].values())
# Count parameters
total_params = sum(p.numel() for p in state_dict["model"].values())
print(f"Total parameters in checkpoint: {total_params}")


dict_values([tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0122,  0.0046,  0.0025,  ...,  0.0089,  0.0141, -0.0041],
        [-0.0033, -0.0115,  0.0183,  ..., -0.0296, -0.0161, -0.0046],
        ...,
        [-0.0050,  0.0798,  0.0376,  ...,  0.0018, -0.0411,  0.0090],
        [ 0.0110,  0.0346,  0.0451,  ..., -0.0286, -0.0257,  0.0001],
        [-0.0036,  0.0008,  0.0154,  ..., -0.0083, -0.0022, -0.0297]]), tensor([0.5947, 0.5459, 0.5615, 0.6245, 0.5239, 0.5171, 0.5322, 0.7100, 0.5923,
        0.6001, 0.6035, 0.6069, 0.6597, 0.6084, 0.6079, 0.6509, 0.6860, 0.5713,
        0.6274, 0.5542, 0.5713, 0.5654, 0.5605, 0.4861, 0.5610, 0.5815, 0.6191,
        0.5620, 0.6021, 0.5728, 0.5068, 0.6465, 0.5845, 0.4473, 0.6099, 0.5684,
        0.6646, 0.5449, 0.5850, 0.5527, 0.5459, 0.6064, 0.5439, 0.6050, 0.6372,
        0.5635, 0.5269, 0.5596, 0.5342, 0.5454, 0.5488, 0.6987, 1.3125, 0.6138,
        0.6240, 0.6304, 0.5625, 0.5200, 0.5654, 0.5488, 0.5498, 0.5537, 

{
  'atoms': 
    ['C', 'O', 'C', 'C', 'Cl', 'C', 'C', 'N', 'C', 'O', 'N', 'C', 'C', 'C', 'S', 'C', 'C', 'O', 'C', 'C', 'Cl', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'], 
  'coordinates': 
    [  
      [
        [ 5.7474146 , -0.42737713, -0.671256  ],
        [ 5.0595884 , -0.873312  ,  0.49939108],
        [ 3.7335892 , -0.5272272 ,  0.44309688],
        [ 3.2894847 ,  0.6755131 ,  0.9917155 ],
        [ 4.3869643 ,  1.7868254 ,  1.733296  ],
        [ 1.9271951 ,  1.0098677 ,  0.9667565 ],
        [ 0.96455485,  0.14713344,  0.41510382],
        [-0.42406547,  0.49487233,  0.39038765],
        [-0.94065493,  1.7353208 ,  0.68675125],
        [-0.37511313,  2.7829528 ,  0.9546822 ],
        [-2.3129926 ,  1.6798112 ,  0.61305684],
        [-2.7886627 ,  0.43308115,  0.0777128 ],
        [-3.8673182 , -0.19075151,  0.95417225],
        [-4.5347543 , -1.2939454 ,  0.15051603],
        [-4.67886   , -0.6208725 , -1.5233577 ],
        [-3.331029  ,  0.5865149 , -1.3439028 ],
        [-1.4680768 , -0.3581401 ,  0.07718275],
        [-1.4170305 , -1.5547268 , -0.18123929],
        [ 1.4297487 , -1.0730952 , -0.10311134],
        [ 2.7914264 , -1.4112226 , -0.08236988],
        [ 3.2577145 , -2.9486456 , -0.7216302 ],
        [ 5.340555  , -0.8908957 , -1.5761046 ],
        [ 5.7169304 ,  0.6633551 , -0.76301974],
        [ 6.794742  , -0.7296767 , -0.5781574 ],
        [ 1.6461692 ,  1.9572424 ,  1.4191216 ],
        [-2.8543859 ,  2.5318658 ,  0.7051557 ],
        [-4.6130214 ,  0.564678  ,  1.2377162 ],
        [-3.45821   , -0.59392905,  1.8880684 ],
        [-5.5192795 , -1.5477054 ,  0.55311584],
        [-3.9281805 , -2.2035532 ,  0.12536196],
        [-2.57242   ,  0.40937346, -2.1129584 ],
        [-3.7551959 ,  1.5826553 , -1.5092808 ],
        [ 0.75317353, -1.7959868 , -0.5500034 ]
      ],
      [
        [ 5.7474146 , -0.42737713, -0.671256  ],
        [ 5.0595884 , -0.873312  ,  0.49939108],
        [ 3.7335892 , -0.5272272 ,  0.44309688],
        [ 3.2894847 ,  0.6755131 ,  0.9917155 ],
        [ 4.3869643 ,  1.7868254 ,  1.733296  ],
        [ 1.9271951 ,  1.0098677 ,  0.9667565 ],
        [ 0.96455485,  0.14713344,  0.41510382],
        [-0.42406547,  0.49487233,  0.39038765],
        [-0.94065493,  1.7353208 ,  0.68675125],
        [-0.37511313,  2.7829528 ,  0.9546822 ],
        [-2.3129926 ,  1.6798112 ,  0.61305684],
        [-2.7886627 ,  0.43308115,  0.0777128 ],
        [-3.8673182 , -0.19075151,  0.95417225],
        [-4.5347543 , -1.2939454 ,  0.15051603],
        [-4.67886   , -0.6208725 , -1.5233577 ],
        [-3.331029  ,  0.5865149 , -1.3439028 ],
        [-1.4680768 , -0.3581401 ,  0.07718275],
        [-1.4170305 , -1.5547268 , -0.18123929],
        [ 1.4297487 , -1.0730952 , -0.10311134],
        [ 2.7914264 , -1.4112226 , -0.08236988],
        [ 3.2577145 , -2.9486456 , -0.7216302 ],
        [ 5.340555  , -0.8908957 , -1.5761046 ],
        [ 5.7169304 ,  0.6633551 , -0.76301974],
        [ 6.794742  , -0.7296767 , -0.5781574 ],
        [ 1.6461692 ,  1.9572424 ,  1.4191216 ],
        [-2.8543859 ,  2.5318658 ,  0.7051557 ],
        [-4.6130214 ,  0.564678  ,  1.2377162 ],
        [-3.45821   , -0.59392905,  1.8880684 ],
        [-5.5192795 , -1.5477054 ,  0.55311584],
        [-3.9281805 , -2.2035532 ,  0.12536196],
        [-2.57242   ,  0.40937346, -2.1129584 ],
        [-3.7551959 ,  1.5826553 , -1.5092808 ],
        [ 0.75317353, -1.7959868 , -0.5500034 ]
      ],
      [
        [ 5.689874  , -0.6729436 , -0.5926651 ],
        [ 5.06305   ,  0.6112394 , -0.5758087 ],
        [ 3.7348907 ,  0.49730274, -0.25296292],
        [ 3.3149593 ,  0.60681254,  1.0724705 ],
        [ 4.446022  ,  0.8436592 ,  2.3585975 ],
        [ 1.9521232 ,  0.5300487 ,  1.3970616 ],
        [ 0.9666086 ,  0.35847554,  0.4101266 ],
        [-0.42305323,  0.2776954 ,  0.74446464],
        [-0.9641589 ,  0.53874356,  1.982348  ],
        [-0.43592116,  0.93435645,  3.0086694 ],
        [-2.3192246 ,  0.30523   ,  1.9382204 ],
        [-2.7934203 ,  0.04984788,  0.6050823 ],
        [-3.6617422 , -1.1997255 ,  0.5356836 ],
        [-4.3783283 , -1.1906176 , -0.8039993 ],
        [-4.8494024 ,  0.5375781 , -1.062506  ],
        [-3.579271  ,  1.232828  ,  0.03758504],
        [-1.4406999 , -0.10193915, -0.11354216],
        [-1.3483069 , -0.49353117, -1.2708981 ],
        [ 1.4091061 ,  0.28345945, -0.92140734],
        [ 2.770574  ,  0.36157563, -1.251269  ],
        [ 3.2073452 ,  0.301506  , -2.9231558 ],
        [ 5.2432613 , -1.3270091 , -1.3488419 ],
        [ 6.7427855 , -0.5267442 , -0.85141385],
        [ 5.6465316 , -1.1530104 ,  0.3905884 ],
        [ 1.6944743 ,  0.5988942 ,  2.4504595 ],
        [-2.8944886 ,  0.5470046 ,  2.7371283 ],
        [-3.073091  , -2.1180065 ,  0.64666253],
        [-4.3987136 , -1.1981632 ,  1.3505465 ],
        [-3.7239864 , -1.5143774 , -1.6182145 ],
        [-5.2607913 , -1.8363307 , -0.7942262 ],
        [-4.094525  ,  1.7913264 ,  0.826355  ],
        [-2.9522424 ,  1.9352188 , -0.52080697],
        [ 0.70976114,  0.18959577, -1.747572  ]
      ], 
      [
        [ 5.8527966 ,  0.1835155 , -0.41091603],
        [ 5.188632  , -0.67537504,  0.5183397 ],
        [ 3.8275752 , -0.5790373 ,  0.37931126],
        [ 3.136659  , -1.4493008 , -0.46330753],
        [ 3.964922  , -2.64692   , -1.3955845 ],
        [ 1.7398154 , -1.3754455 , -0.573774  ],
        [ 0.98793113, -0.44574156,  0.1649254 ],
        [-0.437256  , -0.37067953,  0.04498195],
        [-1.2316266 , -1.2802399 , -0.6200696 ],
        [-0.9400928 , -2.3336287 , -1.162654  ],
        [-2.5395665 , -0.8516854 , -0.5771347 ],
        [-2.7073963 ,  0.2811336 ,  0.29025665],
        [-3.463286  ,  1.4272631 , -0.3681954 ],
        [-4.9332952 ,  1.0372002 , -0.38117784],
        [-5.2094955 ,  0.11470325,  1.1595657 ],
        [-3.445566  , -0.05210027,  1.5830978 ],
        [-1.2442056 ,  0.61647767,  0.584294  ],
        [-0.91704834,  1.6123013 ,  1.2190531 ],
        [ 1.7015941 ,  0.40082577,  1.0299252 ],
        [ 3.0982754 ,  0.32987484,  1.1452627 ],
        [ 3.876145  ,  1.3875434 ,  2.270503  ],
        [ 5.5980425 ,  1.2343538 , -0.23795237],
        [ 6.930284  ,  0.06771144, -0.26072422],
        [ 5.6238256 , -0.0919508 , -1.4457073 ],
        [ 1.2663862 , -2.066054  , -1.2664278 ],
        [-3.2799935 , -1.477108  , -0.8760381 ],
        [-3.107643  ,  1.632152  , -1.3847524 ],
        [-3.3317285 ,  2.3557818 ,  0.20335208],
        [-5.1601224 ,  0.38154864, -1.227648  ],
        [-5.5858235 ,  1.9129096 , -0.43492097],
        [-3.21611   ,  0.6577101 ,  2.3846843 ],
        [-3.2386048 , -1.0636837 ,  1.9462998 ],
        [ 1.1959766 ,  1.1259443 ,  1.6610404 ]
      ],
      [
        [ 5.855724  ,  0.7030209 ,  0.24744147],
        [ 5.1389785 ,  0.4338646 , -0.9592999 ],
        [ 3.8060064 ,  0.24365626, -0.6982895 ],
        [ 2.9115317 ,  1.3117998 , -0.7614863 ],
        [ 3.448129  ,  2.915073  , -1.1242057 ],
        [ 1.5415754 ,  1.1098968 , -0.5339498 ],
        [ 1.020075  , -0.16390419, -0.25263843],
        [-0.3772216 , -0.37213352, -0.01776974],
        [-0.93986434, -1.5379537 ,  0.4564873 ],
        [-0.4108937 , -2.5676682 ,  0.8427109 ],
        [-2.3085837 , -1.3942144 ,  0.49512964],
        [-2.7167783 , -0.04382813,  0.223547  ],
        [-3.784178  ,  0.04709188, -0.85873467],
        [-5.098239  , -0.37540105, -0.21986593],
        [-5.032214  ,  0.23745997,  1.4894487 ],
        [-3.2655833 ,  0.67991436,  1.4490701 ],
        [-1.3792725 ,  0.56502306, -0.2016069 ],
        [-1.287394  ,  1.7155561 , -0.6130999 ],
        [ 1.9346724 , -1.2309697 , -0.22623958],
        [ 3.304858  , -1.0347348 , -0.4552934 ],
        [ 4.338994  , -2.4202762 , -0.44344243],
        [ 5.4969893 ,  1.6177493 ,  0.73091036],
        [ 5.7952423 , -0.13822436,  0.94578695],
        [ 6.907655  ,  0.84954154, -0.01508201],
        [ 0.90515   ,  1.9888537 , -0.57714105],
        [-2.8724113 , -2.094967  ,  0.9638797 ],
        [-3.55434   , -0.5833613 , -1.7256656 ],
        [-3.8683376 ,  1.0762545 , -1.2327403 ],
        [-5.959924  ,  0.03854615, -0.75061333],
        [-5.1937323 , -1.4654623 , -0.1977696 ],
        [-2.7785702 ,  0.37470514,  2.3805783 ],
        [-3.1861227 ,  1.7681044 ,  1.3568267 ],
        [ 1.6080803 , -2.2530124 , -0.05300492]
      ],
      [
        [ 5.8758230e+00, -2.2785266e-01,  6.0796005e-01],
        [ 5.1896324e+00, -1.8071043e-01, -6.4476287e-01],
        [ 3.8317497e+00, -2.1202196e-01, -4.5424923e-01],
        [ 3.1426020e+00, -1.4242655e+00, -4.4971943e-01],
        [ 3.9676235e+00, -2.9300425e+00, -6.5289354e-01],
        [ 1.7486783e+00, -1.4505482e+00, -2.9265797e-01],
        [ 9.9734437e-01, -2.7043048e-01, -1.5615466e-01],
        [-4.2537695e-01, -3.0535221e-01,  3.8573167e-03],
        [-1.1709532e+00, -1.4376656e+00,  2.5406969e-01],
        [-8.1361842e-01, -2.5846460e+00,  4.6863833e-01],
        [-2.5055926e+00, -1.1008291e+00,  2.8276879e-01],
        [-2.7037561e+00,  3.2142612e-01,  2.3982027e-01],
        [-3.6917086e+00,  7.5061160e-01, -8.3660525e-01],
        [-5.0828600e+00,  4.2657715e-01, -3.1400692e-01],
        [-5.0164266e+00,  7.2645938e-01,  1.4769065e+00],
        [-3.2050958e+00,  9.0205348e-01,  1.5583146e+00],
        [-1.2737031e+00,  7.8764087e-01, -3.9411090e-02],
        [-9.9884677e-01,  1.9644095e+00, -2.4230079e-01],
        [ 1.7078853e+00,  9.4076335e-01, -1.9868425e-01],
        [ 3.1023107e+00,  9.7234833e-01, -3.5327169e-01],
        [ 3.8786435e+00,  2.5160668e+00, -4.1832244e-01],
        [ 5.6615238e+00, -1.1561402e+00,  1.1477704e+00],
        [ 5.6267776e+00,  6.3549268e-01,  1.2338766e+00],
        [ 6.9497418e+00, -1.9711891e-01,  4.0132028e-01],
        [ 1.2719960e+00, -2.4271684e+00, -3.0789205e-01],
        [-3.1860311e+00, -1.7788339e+00,  6.0833913e-01],
        [-3.5104651e+00,  2.4793963e-01, -1.7938604e+00],
        [-3.6099908e+00,  1.8289424e+00, -1.0280812e+00],
        [-5.3331532e+00, -6.2521917e-01, -4.8405895e-01],
        [-5.8492794e+00,  1.0472233e+00, -7.8617865e-01],
        [-2.9673951e+00,  1.9662069e+00,  1.6585838e+00],
        [-2.8137653e+00,  3.7559921e-01,  2.4345260e+00],
        [ 1.2056853e+00,  1.8990844e+00, -1.0442278e-01]
      ],
      [
        [ 5.901958  ,  0.1300559 ,  0.11025272],
        [ 5.086718  ,  1.0678467 ,  0.81617516],
        [ 3.7576125 ,  0.8346677 ,  0.570765  ],
        [ 3.1024137 ,  1.4877523 , -0.47263068],
        [ 3.937766  ,  2.5883298 , -1.5119663 ],
        [ 1.7338357 ,  1.2761421 , -0.6975689 ],
        [ 0.97262514,  0.42449865,  0.12162285],
        [-0.42406076,  0.21346612, -0.11437454],
        [-1.1114224 ,  0.6170284 , -1.2392281 ],
        [-0.7057351 ,  1.1369246 , -2.266076  ],
        [-2.4433985 ,  0.30209094, -1.0902312 ],
        [-2.6794584 , -0.50570107,  0.07410166],
        [-3.7854133 ,  0.0452269 ,  0.9642054 ],
        [-5.1055064 , -0.28873283,  0.28653702],
        [-4.858129  , -1.9087707 , -0.49831736],
        [-3.0530999 , -1.9473605 , -0.25590906],
        [-1.298512  , -0.44882175,  0.72994435],
        [-1.0752064 , -0.9502845 ,  1.8254644 ],
        [ 1.6451143 , -0.19567044,  1.1878384 ],
        [ 3.0143232 ,  0.01088408,  1.4156971 ],
        [ 3.741774  , -0.78002024,  2.7704208 ],
        [ 5.6993294 , -0.8974293 ,  0.4302169 ],
        [ 6.94708   ,  0.35631287,  0.34149414],
        [ 5.768293  ,  0.21830562, -0.9729758 ],
        [ 1.2820565 ,  1.8231922 , -1.5209011 ],
        [-3.0690975 ,  0.38468164, -1.8841841 ],
        [-3.7505293 , -0.42195684,  1.9575136 ],
        [-3.6938396 ,  1.1260773 ,  1.1230848 ],
        [-5.932238  , -0.32274184,  1.0013131 ],
        [-5.346698  ,  0.44366875, -0.49015772],
        [-2.5589936 , -2.32221   , -1.1577897 ],
        [-2.8320768 , -2.631355  ,  0.570094  ],
        [ 1.1325148 , -0.86609745,  1.8713869 ]
      ], 
      [
        [ 5.704222  ,  0.57799286,  1.34679   ],
        [ 5.1791105 , -0.50307775,  0.5735715 ],
        [ 3.839435  , -0.3251923 ,  0.33966732],
        [ 2.8895843 , -0.8670829 ,  1.2050108 ],
        [ 3.3568869 , -1.7517792 ,  2.6152778 ],
        [ 1.5190151 , -0.71721554,  0.94398206],
        [ 1.0539384 , -0.04247973, -0.19735526],
        [-0.34502375,  0.11321633, -0.46163374],
        [-0.8879801 ,  0.5804628 , -1.639555  ],
        [-0.34873936,  0.8801693 , -2.6923668 ],
        [-2.256851  ,  0.65094703, -1.5075159 ],
        [-2.7121499 ,  0.0490635 , -0.2851197 ],
        [-3.649736  ,  0.949536  ,  0.5077362 ],
        [-5.0013514 ,  0.9018325 , -0.18804051],
        [-5.1723123 , -0.7961886 , -0.8114765 ],
        [-3.442317  , -1.2733535 , -0.4983042 ],
        [-1.3730645 , -0.18935138,  0.41477367],
        [-1.3022333 , -0.60116553,  1.5667323 ],
        [ 2.0285077 ,  0.46798486, -1.0719956 ],
        [ 3.4001465 ,  0.32225078, -0.814704  ],
        [ 4.517477  ,  0.9631856 , -1.9680595 ],
        [ 6.7706    ,  0.38949272,  1.5025231 ],
        [ 5.602658  ,  1.5332328 ,  0.82114536],
        [ 5.2242336 ,  0.6381995 ,  2.3291657 ],
        [ 0.83435744, -1.1703107 ,  1.6551086 ],
        [-2.8294818 ,  0.82059777, -2.3272977 ],
        [-3.7517145 ,  0.5881189 ,  1.5397723 ],
        [-3.282932  ,  1.9806536 ,  0.57177454],
        [-5.8186555 ,  1.1521604 ,  0.49379316],
        [-5.0326543 ,  1.5916992 , -1.0371252 ],
        [-3.050356  , -1.8479958 , -1.3434088 ],
        [-3.4170482 , -1.9078307 ,  0.39378378],
        [ 1.754428  ,  1.0040138 , -1.9766493 ]
      ],
      [
        [ 5.7639165 ,  1.071723  ,  0.37929258],
        [ 5.188262  , -0.19124791,  0.719658  ],
        [ 3.8427486 , -0.19753733,  0.4538792 ],
        [ 3.364276  , -0.64628726, -0.7767373 ],
        [ 4.4406867 , -1.1664643 , -2.025821  ],
        [ 1.9852965 , -0.6897246 , -1.0323924 ],
        [ 1.0414993 , -0.30536407, -0.06447193],
        [-0.36491576, -0.34656087, -0.3318253 ],
        [-0.95088774, -0.92450076, -1.4378278 ],
        [-0.4531061 , -1.5615299 , -2.3518436 ],
        [-2.3114088 , -0.7183591 , -1.3858831 ],
        [-2.7228677 , -0.14918661, -0.13266434],
        [-3.6104085 ,  1.0767931 , -0.29935184],
        [-4.991569  ,  0.5726547 , -0.68856233],
        [-5.216949  , -0.9872238 ,  0.21540563],
        [-3.4829428 , -1.1294564 ,  0.75544304],
        [-1.3586814 ,  0.17496186,  0.47869912],
        [-1.2445565 ,  0.7892485 ,  1.5328544 ],
        [ 1.5447237 ,  0.11475905,  1.1782979 ],
        [ 2.9227302 ,  0.15845153,  1.4395366 ],
        [ 3.4364588 ,  0.6495351 ,  3.0158715 ],
        [ 5.6551037 ,  1.2870873 , -0.6888266 ],
        [ 5.3280973 ,  1.8833092 ,  0.9712943 ],
        [ 6.832858  ,  1.0233048 ,  0.6072514 ],
        [ 1.6808976 , -1.0260764 , -2.0198405 ],
        [-2.9162872 , -1.2119212 , -2.0333767 ],
        [-3.2245057 ,  1.7717644 , -1.0542977 ],
        [-3.6733012 ,  1.6391925 ,  0.64185584],
        [-5.0485435 ,  0.3669868 , -1.7620124 ],
        [-5.7743173 ,  1.2909979 , -0.4300372 ],
        [-3.13743   , -2.162399  ,  0.6472771 ],
        [-3.4292305 , -0.8550468 ,  1.8141384 ],
        [ 0.88435334,  0.39811662,  1.9927734 ]
      ], 
      [
        [ 5.8897905 , -0.40005818, -0.0246694 ],
        [ 5.16781   ,  0.68079937, -0.61870974],
        [ 3.8161721 ,  0.4629624 , -0.53659433],
        [ 3.1359627 , -0.18008456, -1.5701736 ],
        [ 3.9653978 , -0.75220513, -2.9751713 ],
        [ 1.7473111 , -0.3676408 , -1.5005217 ],
        [ 0.9919148 ,  0.09732047, -0.41010833],
        [-0.42537668, -0.09800387, -0.34612244],
        [-1.1555351 , -0.9128042 , -1.1849269 ],
        [-0.78417677, -1.6734517 , -2.064017  ],
        [-2.4911425 , -0.8027984 , -0.8693355 ],
        [-2.700432  , -0.04434272,  0.3326904 ],
        [-3.7217073 ,  1.071905  ,  0.15968758],
        [-5.0950584 ,  0.41821122,  0.16029555],
        [-4.979624  , -0.9649406 ,  1.3330193 ],
        [-3.1679852 , -0.8941371 ,  1.5100352 ],
        [-1.2822447 ,  0.474091  ,  0.5783498 ],
        [-1.0215626 ,  1.2577245 ,  1.4835889 ],
        [ 1.6924303 ,  0.7730767 ,  0.60295576],
        [ 3.0816166 ,  0.961234  ,  0.53912055],
        [ 3.8449187 ,  1.8281924 ,  1.8258777 ],
        [ 5.69751   , -1.3443565 , -0.54463834],
        [ 6.957132  , -0.17665182, -0.11308775],
        [ 5.6526556 , -0.503316  ,  1.0394105 ],
        [ 1.2772574 , -0.8672723 , -2.3435292 ],
        [-3.157228  , -1.4482317 , -1.2797356 ],
        [-3.653069  ,  1.7899805 ,  0.9878151 ],
        [-3.5643427 ,  1.6422589 , -0.76324207],
        [-5.342709  ,  0.01950167, -0.82841074],
        [-5.878989  ,  1.1195407 ,  0.45885214],
        [-2.9350026 , -0.42951933,  2.4737935 ],
        [-2.7481046 , -1.9049194 ,  1.5022012 ],
        [ 1.1864115 ,  1.1679354 ,  1.479027  ]
      ]
    ], 
  'smi': 
    'COc1c(Cl)cc([N+]2C(=O)N[C@]3(CCSC3)C2=O)cc1Cl'
}